In [1]:
import os, sys 
import pandas as pd
import pickle as pk
from pathlib2 import Path
from evaluate import autoGrader
import numpy as np
from sklearn import metrics
#pip install pathlib2
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
fd = Path(os.getcwd())
fd_data = fd/'data'

# Step 1: getting the data
*load training data*

标准化后提高到0.84，但标准化数据对tree-method其实影响不大

In [3]:
d_train = pd.read_csv(fd_data/'training_data.csv',parse_dates = ['t_ob'])  #parse_dates: parse as date，自动改为时间，不需要再用pd.to_datetime转换
d_train.head(1)

,t_ob,风向,PM10,PM2.5,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速
0,2020-03-01 00:10:00,338.0,32.7,23.4,25.0,31.35,3.74,38.23,28.54,27.04,0.98,0.76,1015.2,91.0,9.3,1.4


In [4]:
# 检查数据，很干净，没有缺失，不需要额外清洗
d_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11437 entries, 0 to 11436
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   t_ob    11437 non-null  datetime64[ns]
 1   风向      11437 non-null  float64       
 2   PM10    11437 non-null  float64       
 3   PM2.5   11437 non-null  float64       
 4   室内温度    11437 non-null  float64       
 5   室内湿度    11437 non-null  float64       
 6   SO2     11437 non-null  float64       
 7   O3      11437 non-null  float64       
 8   NOx     11437 non-null  float64       
 9   NO2     11437 non-null  float64       
 10  NO      11437 non-null  float64       
 11  CO      11437 non-null  float64       
 12  气压      11437 non-null  float64       
 13  大气湿度    11437 non-null  float64       
 14  大气温度    11437 non-null  float64       
 15  风速      11437 non-null  float64       
dtypes: datetime64[ns](1), float64(15)
memory usage: 1.4 MB


In [20]:
#import matplotlib.pyplot as plt
#x=d_train['t_ob']
#y=d_train['PM2.5']
#plt.plot(x,y)

In [5]:
# Convert the time 't_ob' to the time since 2020-03-01 00:00:00
# This variable was used to take account of the trend over time

from datetime import datetime, date
time_original = '2020-03-01 00:00:00'
time_original_1 = datetime.strptime(time_original,"%Y-%m-%d %H:%M:%S")
d_train['time'] = d_train['t_ob']-time_original_1 
d_train['duration']=d_train['time'].astype('timedelta64[s]')/60

In [6]:
d_train.head(1)

,t_ob,风向,PM10,PM2.5,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,time,duration
0,2020-03-01 00:10:00,338.0,32.7,23.4,25.0,31.35,3.74,38.23,28.54,27.04,0.98,0.76,1015.2,91.0,9.3,1.4,0 days 00:10:00,10.0


*load features for closed test data*

In [7]:
X_ind_test = pd.read_csv(fd_data/'closed_test_X.csv', parse_dates = ['t_ob'])
X_ind_test.head(2)

,t_ob,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速
0,2020-06-01 00:00:00,129.500000,50.266667,26.851667,52.270000,5.965000,180.263333,7.696667,6.688333,0.656667,0.653333,1006.800000,76.333333,24.916667,1.483333
1,2020-06-01 00:10:00,134.333333,49.266667,26.870000,54.566667,6.213333,181.400000,7.388333,6.356667,0.671667,0.640000,1006.716667,74.866667,24.883333,1.466667


In [8]:
X_ind_test['time'] = X_ind_test['t_ob']-time_original_1
X_ind_test['duration']=X_ind_test['time'].astype('timedelta64[s]')/60

In [9]:
X_ind_test.head(2)

,t_ob,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,time,duration
0,2020-06-01 00:00:00,129.500000,50.266667,26.851667,52.270000,5.965000,180.263333,7.696667,6.688333,0.656667,0.653333,1006.800000,76.333333,24.916667,1.483333,92 days 00:00:00,132480.0
1,2020-06-01 00:10:00,134.333333,49.266667,26.870000,54.566667,6.213333,181.400000,7.388333,6.356667,0.671667,0.640000,1006.716667,74.866667,24.883333,1.466667,92 days 00:10:00,132490.0


In [10]:
# 查看一下数据情况，气压/duration的最大、最小值差了很多，考虑是否标准化
d_train.describe()

,风向,PM10,PM2.5,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,time,duration
count,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,11437.000000,1.143700e+04,11437,11437.000000
mean,139.058289,62.581067,34.589047,26.508754,32.015774,8.037296,101.850400,30.551121,27.311207,2.113133,0.801241,1012.535302,65.696977,16.259896,2.461683e+00,42 days 23:56:45.001311532,61916.750022
std,77.974586,28.772283,18.746774,1.639887,9.800393,3.071383,52.339394,15.051652,12.958800,2.449024,0.256533,6.294068,21.037003,5.811326,1.313624e+00,26 days 18:18:18.049959448,38538.300833
min,0.000000,6.100000,4.016667,19.296667,13.613333,2.310000,-9.226667,7.128333,6.446667,0.140000,-0.083333,994.216667,18.850000,0.466667,4.450144e-15,0 days 00:10:00,10.000000
25%,83.333333,40.316667,20.916667,25.913333,24.390000,5.728333,64.030000,20.561667,18.318333,1.083333,0.630000,1007.516667,49.316667,11.733333,1.433333e+00,19 days 21:30:00,28650.000000
50%,135.666667,57.800000,30.583333,26.800000,29.803333,7.580000,94.043333,26.751667,24.175000,1.391667,0.761667,1013.083333,65.783333,16.166667,2.316667e+00,40 days 12:10:00,58330.000000
75%,175.166667,80.133333,43.983333,27.558333,37.168333,9.588333,133.053333,36.636667,33.001667,2.270000,0.935000,1017.266667,83.333333,20.616667,3.300000e+00,70 days 02:20:00,100940.000000
max,354.000000,191.933333,111.016667,38.848333,59.465000,26.795000,510.740000,152.298333,104.753333,47.725000,2.743333,1027.050000,99.000000,30.200000,7.550000e+00,90 days 00:00:00,129600.000000


*build model*

# Step 2: create X,Y and train-test split

Note: we should split the dataset first, and then apply the standardization method

Reason: Don't forget that testing data points represent real-world data. Feature normalization (or data standardization) of the explanatory (or predictor) variables is a technique used to center and normalise the data by subtracting the mean and dividing by the variance. If you take the mean and variance of the whole dataset you'll be introducing future information into the training explanatory variables (i.e. the mean and variance).

Therefore, you should perform feature normalisation over the training data. Then perform normalisation on testing instances as well, but this time using the mean and variance of training explanatory variables. In this way, we can test and evaluate whether our model can generalize well to new, unseen data points.

In [10]:
d_train.shape

(11437, 18)

In [11]:
# create X and Y and run the basic regression model
y = d_train['PM2.5']  # y:PM2.5
X= d_train.drop(['PM2.5','t_ob','time'], axis=1)
#X= d_train.drop(['PM2.5','t_ob','time'], axis=1) #x:other variable

In [12]:
X.head(2)

,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,duration
0,338.0,32.70,25.0,31.35,3.74,38.230,28.54,27.04,0.98,0.760,1015.20,91.0,9.3,1.4,10.0
1,173.5,31.85,25.0,31.39,3.94,36.265,28.05,26.55,0.98,0.745,1015.15,91.2,9.3,1.0,20.0


In [13]:
y.head(1)

0    23.4
Name: PM2.5, dtype: float64

# build a more complex model-random forest

In [14]:
# 80% train, 20% test  
# split the dataset in training set and test set
from sklearn.model_selection import train_test_split
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.2, random_state=1)
#X_val_rf, X_test_rf, y_val_rf, y_test_rf = train_test_split(X_test_rf, y_test_rf, test_size=0.5, random_state=1)

In [15]:
# 计算均值和方差，用于之后的标准化
training_mean = np.mean(X_train_rf,axis=0)
training_std=np.std(X_train_rf,axis=0)

In [16]:
X_train_rf.columns

Index(['风向', 'PM10', '室内温度', '室内湿度', 'SO2', 'O3', 'NOx', 'NO2', 'NO', 'CO',
       '气压', '大气湿度', '大气温度', '风速', 'duration'],
      dtype='object')

In [17]:
data_columns=['风向', 'PM10', '室内温度', '室内湿度', 'SO2', 'O3', 'NOx', 'NO2', 'NO',
       'CO', '气压', '大气湿度', '大气温度', '风速', 'duration']
# 标准化training data
for i in data_columns:
    X_train_rf[i]=(X_train_rf[i]-training_mean[i])/training_std[i]
# 标准化test data
for i in data_columns:
    X_test_rf[i]=(X_test_rf[i]-training_mean[i])/training_std[i]

<ipython-input-17-0988a327dae6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_rf[i]=(X_train_rf[i]-training_mean[i])/training_std[i]
<ipython-input-17-0988a327dae6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_rf[i]=(X_test_rf[i]-training_mean[i])/training_std[i]


In [19]:
'''
for col in ['风向', 'PM10', 'PM2.5', '室内温度', '室内湿度', 'SO2', 'O3', 'NOx',
       'NO2', 'NO', 'CO', '气压', '大气湿度', '大气温度', '风速']:
    stdscl = StandardScaler()
    d_train[col] = stdscl.fit_transform(d_train[col].values.reshape(-1,1))'''

In [18]:
d_train.head(1)

,t_ob,风向,PM10,PM2.5,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,time,duration
0,2020-03-01 00:10:00,338.0,32.7,23.4,25.0,31.35,3.74,38.23,28.54,27.04,0.98,0.76,1015.2,91.0,9.3,1.4,0 days 00:10:00,10.0


In [19]:
X_train_rf.head(1)

,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,duration
3283,-0.924784,-0.946577,0.861878,-1.304804,-0.442527,-0.446259,0.560918,0.807657,-0.567475,-1.245937,0.606049,-1.366499,-0.16642,0.786297,-0.761526


In [20]:
# random forest
# 1.import the model
from sklearn.ensemble import RandomForestRegressor

In [21]:
# 2.initialize the random forest model
# 之前的方法training set和test set上R2都比较高，0.99+，但是最终在X_ind_test上计算的R2只有0.83+
# 考虑是否为过拟合问题，增加min_samples_leaf进行修剪，先设为5
# 未修剪之前：training set 0.99815, test set 0.98692
# 修剪之后：training set 0.99645 test set 0.98329
rf = RandomForestRegressor(n_estimators=100,max_depth=30,max_features='sqrt')  #n_estimators: hyper-parameter, represent the number of trees, default=100,test it
#the number of trees in the forest (n_estimators) and the number of features considered for splitting at each leaf node (max_features)
#n_estimators = number of trees in the foreset
#max_features = max number of features considered for splitting a node, auto/sqrt/log2
#max_depth = max number of levels in each decision tree
#min_samples_split = min number of data points placed in a node before the node is split
#min_samples_leaf = min number of data points allowed in a leaf node
#bootstrap = method for sampling data points (with or without replacement)

In [22]:
#3.fit the model to the training set
model_rf=rf.fit(X_train_rf,y_train_rf)
#4. evaluate the performance of the model
rf.score(X_train_rf,y_train_rf)

0.9987394779463152

In [23]:
# test n_estimators as 80,120 manually
rf2 = RandomForestRegressor(n_estimators=80,max_depth=50,max_features='sqrt')
# fit the model to the training set
model_rf2=rf2.fit(X_train_rf,y_train_rf)
print(rf2.score(X_train_rf, y_train_rf))

0.9986057063699388


#### hyper-parameter tuning

###### method 2: cross validation

In [65]:
# 第一次cross validation，最优结果都在区间端点处取得
# 可能不是最优，但数量过多，时间很久，所以在第一次获得的结果上，扩大范围进行第二次cross validation
import time
from sklearn import model_selection
start_cv=time.time()
# values of n_estimators and max_depth
parameters = {'n_estimators':[80,100,120,150,180,200,220,240], #[122,123,124,125,126,127,128,129,130],
              'max_depth':[24,25,26,27,28,29],#[19,21,23,25,27,29,31,33],
              'max_features':['auto','sqrt','log2'],
             'min_samples_leaf':[1,5,10]}

# initialize the random forest model
rf = RandomForestRegressor()

#Exhaustive search over specified parameter values for an estimator
clf = model_selection.GridSearchCV(rf, parameters,cv=5)

# fit the model on the training set
clf.fit(X_train_rf, y_train_rf)

# we can query the best parameter value and its accuracy score
print ("The best parameter value is: ")
print (clf.best_params_)
print ("The best score is: ")
print (clf.best_score_)
end_cv=time.time()

KeyboardInterrupt: 

In [24]:
# 第二次cross validation，改变n_estimators和max_depth的数值，240与29作为中间值
import time
from sklearn import model_selection
start_cv=time.time()
# values of n_estimators and max_depth
parameters = {'n_estimators':[200,220,230,240,250,260,270,280,290,300], #[122,123,124,125,126,127,128,129,130],
              'max_depth':[26,27,28,29,30,31,32,33,34,35],#[19,21,23,25,27,29,31,33],
              'max_features':['auto','sqrt','log2']}

# initialize the random forest model
rf = RandomForestRegressor()

#Exhaustive search over specified parameter values for an estimator
clf = model_selection.GridSearchCV(rf, parameters,cv=5)

# fit the model on the training set
clf.fit(X_train_rf, y_train_rf)

# we can query the best parameter value and its accuracy score
print ("The best parameter value is: ")
print (clf.best_params_)
print ("The best score is: ")
print (clf.best_score_)
end_cv=time.time()

The best parameter value is: 
{'max_depth': 33, 'max_features': 'auto', 'n_estimators': 230}
The best score is: 
0.9882540412673163


In [26]:
clf.best_params_

{'max_depth': 33, 'max_features': 'auto', 'n_estimators': 230}

In [39]:
rf_final_cv = RandomForestRegressor(n_estimators = clf.best_params_['n_estimators'],max_depth=clf.best_params_['max_depth'],max_features=clf.best_params_['max_features']) #把通过上面测试得到的最优值带进去
rf_final_cv.fit(X_train_rf, y_train_rf) 

print('The score on the training data:')
print(rf_final_cv.score(X_train_rf, y_train_rf))

The score on the training data:
0.9989692887522308


# use the final model for prediction

In [56]:
# get final estimate
rf_final_cv.score(X_test_rf, y_test_rf)

0.9933567474723087

In [25]:
X_ind_test.head(2)

,t_ob,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,time,duration
0,2020-06-01 00:00:00,129.500000,50.266667,26.851667,52.270000,5.965000,180.263333,7.696667,6.688333,0.656667,0.653333,1006.800000,76.333333,24.916667,1.483333,92 days 00:00:00,132480.0
1,2020-06-01 00:10:00,134.333333,49.266667,26.870000,54.566667,6.213333,181.400000,7.388333,6.356667,0.671667,0.640000,1006.716667,74.866667,24.883333,1.466667,92 days 00:10:00,132490.0


In [26]:
# x for prediction
# training set和test set是标准化过的，用到新的数据集上怎么处理
x_for_prediction=X_ind_test.drop(['t_ob','time'], axis=1) 
#x_for_prediction=X_ind_test.drop(['t_ob','time'], axis=1) 
x_for_prediction.head(1)

,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,duration
0,129.5,50.266667,26.851667,52.27,5.965,180.263333,7.696667,6.688333,0.656667,0.653333,1006.8,76.333333,24.916667,1.483333,132480.0


In [27]:
for i in data_columns:
    x_for_prediction[i]=(x_for_prediction[i]-training_mean[i])/training_std[i]

In [178]:
'''
for col in ['风向', 'PM10', '室内温度', '室内湿度', 'SO2', 'O3', 'NOx',
       'NO2', 'NO', 'CO', '气压', '大气湿度', '大气温度', '风速', 'duration']:
    stdscl = StandardScaler()
    x_for_prediction[col] = stdscl.fit_transform(x_for_prediction[col].values.reshape(-1,1))'''

In [28]:
x_for_prediction

,风向,PM10,室内温度,室内湿度,SO2,O3,NOx,NO2,NO,CO,气压,大气湿度,大气温度,风速,duration
0,-0.115908,-0.431478,0.206668,2.058730,-0.682613,1.477192,-1.533499,-1.601987,-0.614970,-0.586760,-0.908456,0.504777,1.489458,-0.744894,1.831749
1,-0.053687,-0.466321,0.217930,2.292546,-0.601311,1.498694,-1.554201,-1.627749,-0.608590,-0.639495,-0.921718,0.435089,1.483708,-0.757654,1.832010
2,0.105085,-0.510456,0.157528,2.313926,-0.605676,1.513796,-1.567070,-1.642896,-0.607881,-0.646086,-0.934980,0.362233,1.489458,-0.859733,1.832270
3,0.124395,-0.526136,0.147290,2.092835,-0.486724,1.466409,-1.540549,-1.611567,-0.610008,-0.626311,-0.948242,0.351939,1.489458,-0.987333,1.832530
4,0.107230,-0.538911,0.187217,2.035144,-0.424520,1.435984,-1.533387,-1.603799,-0.607881,-0.665862,-0.958852,0.359066,1.480833,-1.051132,1.832791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2934,-0.918347,-1.889086,1.690103,2.196678,-2.234445,-0.334207,-0.508000,-0.566952,-0.072674,-1.259121,-1.693559,1.311729,0.951872,0.582138,2.596790
2935,-0.890455,-1.897797,1.687032,2.151374,-2.181517,-0.315290,-0.504419,-0.559314,-0.086143,-1.272304,-1.704169,1.268174,0.986370,0.582138,2.597050
2936,-0.821797,-1.901281,1.691127,2.034296,-2.079480,-0.310372,-0.511469,-0.556077,-0.126550,-1.298672,-1.709474,1.232539,1.009368,0.505578,2.597311
2937,-0.791759,-1.900700,1.680889,1.815920,-2.041285,-0.294576,-0.537431,-0.572648,-0.174045,-1.311855,-1.706821,1.210365,1.032366,0.518338,2.597571


In [185]:
# 最终结果如何处理
'''
mean=X_ind_test['prediction'].mean()
std_value=X_ind_test['prediction'].std()
std_value
X_ind_test['final_prediction']=std_value* X_ind_test['prediction']+mean
X_ind_test['prediction']
X_ind_test['final_prediction']'''

In [61]:
X_ind_test['prediction'] = rf_final_cv.predict(x_for_prediction)
result=X_ind_test[['t_ob','prediction']] #提取出需要的列
result = result.rename(columns={ 'prediction': 'PM2.5'})  #修改列名
result.to_csv('submission_rf.csv',index=False)  #保存文件

In [62]:
# R2 improved a little bit
# max residual is big, outlier?
# try to tune more hyper-parameters, like max_depth
autoGrader(fd/'submission_rf.csv')

[Enter] to confirm your submission file is placed at D:\code_own\CodingTest-SunyuWang\to_players\submission_rf.csv)

Submission time: 2022-05-23 11:08:27.793620
 - RMSE: 4.878
 - R sq.: 0.839
 - Max. abs. residual: 25.421
 - Min. abs. residual: 0.005
 - Logging results. [Enter] to make sure 'submission_history.csv' is closed: 
 - Notes for this submission: 
 ! Results logged in 'submission_history.csv'.
 ! Submission file saved to '/old_submissions/submission_2022_05_23_11_08_27.csv'.
Evaluation Successful. Press to quit...


### GBRT

In [29]:
# 1. import the model
from sklearn.ensemble import GradientBoostingRegressor 
# learning rate=0.1, shrink the contribution of each tree
# n_estimators=100, the number of boosting stages to perform
# Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance.

In [31]:
?gbrt

In [32]:
# 2. initialize the model
# learning rate: shrink the contribution of each tree
# n_estimators: the number of boosting stages, GBRT一般对overfit较为robust，该值越大，一般model performance越好
# max_depth: 每棵regression tree的深度
gbrt = GradientBoostingRegressor( max_depth=10, n_estimators = 250, learning_rate = 0.1) 

In [34]:
# 3. fit the model
gbrt.fit(X_train_rf, y_train_rf)
# 4. evaluate the model performance
gbrt.score(X_train_rf, y_train_rf)

0.9999895458085911

In [36]:
import time
from sklearn import model_selection
start_cv=time.time()
# values of n_estimators and max_depth
parameters = {'n_estimators':[150,200,250,300,350], #[122,123,124,125,126,127,128,129,130],
              'max_depth':[5,10,15,20,25],#[19,21,23,25,27,29,31,33],
             'learning_rate':[0.05,0.1,0.15]}

# initialize the random forest model
gbrt = GradientBoostingRegressor()

#Exhaustive search over specified parameter values for an estimator
clf = model_selection.GridSearchCV(gbrt, parameters)

# fit the model on the training set
clf.fit(X_train_rf, y_train_rf)

# we can query the best parameter value and its accuracy score
print ("The best parameter value is: ")
print (clf.best_params_)
print ("The best score is: ")
print (clf.best_score_)
end_cv=time.time()

The best parameter value is: 
{'learning_rate': 0.15, 'max_depth': 5, 'n_estimators': 350}
The best score is: 
0.992353711327042


In [38]:
gbrt_final = GradientBoostingRegressor(n_estimators = clf.best_params_['n_estimators'],max_depth=clf.best_params_['max_depth'],learning_rate=clf.best_params_['learning_rate']) #把通过上面测试得到的最优值带进去
gbrt_final.fit(X_train_rf, y_train_rf)

GradientBoostingRegressor(learning_rate=0.15, max_depth=5, n_estimators=350)

In [39]:
gbrt_final.score(X_train_rf, y_train_rf)

0.9988513469806678

In [41]:
gbrt_final.score(X_test_rf, y_test_rf)

0.9945937509584963

In [46]:
X_ind_test['prediction_gbrt'] = gbrt_final.predict(x_for_prediction)
result_gbrt=X_ind_test[['t_ob','prediction_gbrt']] #提取出需要的列
result_gbrt= result_gbrt.rename(columns={ 'prediction_gbrt': 'PM2.5'})  #修改列名
result_gbrt.to_csv('submission_gbrt.csv',index=False)  #保存文件

In [47]:
result_gbrt

,t_ob,PM2.5
0,2020-06-01 00:00:00,35.148101
1,2020-06-01 00:10:00,35.110478
2,2020-06-01 00:20:00,34.148238
3,2020-06-01 00:30:00,31.627540
4,2020-06-01 00:40:00,32.181889
...,...,...
2934,2020-06-21 09:40:00,7.091289
2935,2020-06-21 09:50:00,6.997168
2936,2020-06-21 10:00:00,7.152962
2937,2020-06-21 10:10:00,6.810447


In [48]:
autoGrader(fd/'submission_gbrt.csv')

[Enter] to confirm your submission file is placed at D:\code_own\CodingTest-SunyuWang\to_players\submission_gbrt.csv)

Submission time: 2022-06-06 17:31:23.518892
 - RMSE: 5.391
 - R sq.: 0.803
 - Max. abs. residual: 30.891
 - Min. abs. residual: 0.002
 - Logging results. [Enter] to make sure 'submission_history.csv' is closed: 
 - Notes for this submission: 
 ! Results logged in 'submission_history.csv'.
 ! Submission file saved to '/old_submissions/submission_2022_06_06_17_31_23.csv'.
Evaluation Successful. Press to quit...


gbrt得到的最终测试结果不如random forest，但在training set/test set上表现很好